In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [ ]:
# the website to be scrapped
website="https://www.instahyre.com/search-jobs/"

# get request to the url
opts=Options()
driver=webdriver.Chrome(opts)
driver.get(website)

# creating empty list for the data
data=[]

In [ ]:
# taking total no of pages to be repeated for data collection
pages= 500

# running for loop for different pages upto n pages
for i in range(pages):
    print("Page", i+1)

    # making soup
    html_code=driver.page_source
    soup=BeautifulSoup(html_code,'html.parser')
    
    # all rows in one page
    rows=soup.find_all(class_='row cursor-pointer')
    
    # run loop for each row to get the data by clicking on the view tab
    for row in rows:
        link=row.find('a')['href']

        #printing each link of view tab
        print("directing to link :", link) 
        
        # taking content_driver as a driver for content for each row 
        content_driver = webdriver.Chrome(opts)
        content_driver.get(link)
        
        # making soup for content
        content_html_code = content_driver.page_source
        content_soup = BeautifulSoup(content_html_code, 'html.parser')
        
        # extracting required data from each content page
        
        # link itself contain the job_id https://www.instahyre.com/job-261410-admin-at-indihood-bangalore/
        try:
            job_id = int(link.split('-')[1])
        except:
            job_id = None
        
        # text of tag is itself the job_title
        try:
            job_title = content_soup.find("h1").text.strip()
        except:
            job_title = None
        
        #text of tag is itself the company_name
        try:
            company_name = content_soup.find(class_="company-name").text.strip()
        except:
            company_name = None
        
        #text of tag is itself the experience
        try:
            experience = content_soup.find(class_="experience").text.strip()
        except:
            experience = None
        
        # text of tag is itself the job location
        try:
            job_locations = content_soup.find(class_="job-locations").text.strip().replace(experience, "")
        except:
            job_locations = None
        
        # text of tag is itself the job hr_name
        try:
            HR_name = content_soup.find(class_="rec-name").text.strip()
        except:
            HR_name = None
        
        # text of tag in respective attibute
        try:
            founded_in = content_soup.find("div", {"ng-if": "employer.company_founded"}).text.strip()
        except:
            founded_in = None
        
        # text of tag in respective attibute
        try:
            employees_count = content_soup.find("div", {"ng-if": "employer.employee_count"}).text.strip()
        except:
            employees_count = None
        # there are multiple skills in their tag
        try:
            skill = content_soup.find(class_="tags job-description-skills")
        except:
            skill = None

        if skill and skill.children:
            skill=skill.children
        else:
            skill=[skill]
        
        skills_list = []
        
        if skill is not None:
            for j in skill:
                try:
                    j = j.text.strip()
                except:
                    pass
                if (j and j != "\\n"):
                    skills_list.append(j)
                    
        if skills_list is None:
            skill="N/A"
        else:
            skill= ", ".join(skills_list)
            
        # appending extracted data to data list   
        data.append({"company_name" : company_name, "job_title" : job_title, "job_locations" : job_locations, 
                         "experience" : experience,"hr_name" : HR_name,"founded_in" : founded_in,
                     "employees_count" : employees_count, "skills" : skill, "job_id" : job_id })
        
        # Return to the previous page
        content_driver.back()
        content_driver.quit()
        
    # again clicking the view tab for another row
    next_page = driver.find_element(By.CSS_SELECTOR, '[ng-click="nextPage()"]').click()
    
# printing no of data and data    
print(len(data))
print("data",data)

# converting list into datframe
df = pd.DataFrame(data)
print(df)
           
df.to_excel('G:\My Drive\Projects\Instahyre Job Analytics\Anmol_try_2.xlsx', columns=['company_name', 'job_title', 'job_locations', 'experience', 'hr_name', 
                                                    'founded_in', 'employees_count', 'skills','job_id'], index=False)   